In [87]:
let input = "Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green";

In [79]:
use std::fs;
let input = fs::read_to_string(".day2-input-p1.txt").unwrap();

In [95]:
use std::str::FromStr;
use core::cmp::Ordering;

#[derive(Debug)]
struct Reveal {
    green: i32,
    blue: i32,
    red: i32
}

#[derive(Debug)]
struct Game {
    id: i32,
    reveals: Vec<Reveal>
}

#[derive(Debug)]
struct ParseError;

impl FromStr for Reveal {
    type Err = ParseError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        
        let parts = s.split(", ")
            .map(|e| e.split_once(" "))
            .filter_map(|e| e)
            .collect::<Vec<_>>();
        
        let mut red: i32 = 0;
        let mut green: i32 = 0;
        let mut blue: i32 = 0;

        for (count, color) in parts {
            let c = count.parse::<i32>().map_err(|_| ParseError)?;
            match color {
                "red" => red = c,
                "green" => green = c,
                "blue" => blue = c,
                _ => println!("how do i error?")
            }
        }
        Ok(Reveal { red: red, green: green, blue: blue})
    }
}

impl FromStr for Game {
    type Err = ParseError;
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let (id, realstr) = s
            .strip_prefix("Game ")
            .and_then(|s| s.split_once(": "))
            .ok_or(ParseError)?;
        
        let id_fromstr = id.parse::<i32>().map_err(|_| ParseError)?;
        //println!("{}",realstr);
        
        let reveals: Result<Vec<Reveal>, _> = realstr
            .split("; ")
            .map(Reveal::from_str)
            .collect();
        
        Ok(Game { id: id_fromstr, reveals: reveals? })
    }
}

fn fewest(game: &Game) -> Reveal {
    let mut red: i32 = 0;
    let mut green: i32 = 0;
    let mut blue: i32 = 0;
    
    for reveal in game.reveals.iter() {
        red = if reveal.red > red { reveal.red } else { red };
        green = if reveal.green > green { reveal.green } else { green };
        blue = if reveal.blue > blue { reveal.blue } else { blue };
    }
    
    Reveal {
        red: red,
        green: green,
        blue: blue
    }
}

fn pow(reveal: &Reveal) -> i32 {
    reveal.red * reveal.blue * reveal.green
}

fn could_roll(reveal: &Reveal, key: &Reveal) -> bool {
    return reveal.red < key.red 
        && reveal.green < key.green 
        && reveal.blue < key.blue
}

In [96]:
let key: Reveal = Reveal::from_str("12 red, 13 green, 14 blue").unwrap();
let games: Vec<Game> = input.lines().map(Game::from_str).filter_map(|x| x.ok()).collect();

In [97]:
let mut total = 0;
let mut powtotal = 0;
for game in games.iter() {
    
    if game.reveals.iter().all(|r| could_roll(r, &key)) {
        total += game.id;
    }
    
    powtotal += pow(&fewest(&game));
}
println!("total is {:?}", total);
println!("powtotal is {:?}", powtotal);

total is 8
powtotal is 2286
